In [2]:
!pip install gurobipy

     |████████████████████████████████| 11.5 MB 4.8 MB/s 


In [3]:
import numpy as np
import pandas as pd
import gurobipy as gp
import tensorflow as tf

In [4]:
# problem 1
nsim=100000
sumvec = np.zeros(nsim)
for s in range(nsim):
    sumnow = 0
    for roll in range(5):
        side = np.random.choice([1,2,3,4],p=[0.1,0.2,0.3,0.4])
        sumnow += side
        if (side == 1) | (side == 3):
            break
    sumvec[s] = sumnow
np.mean(sumvec)

6.91633

In [ ]:
# problem 2

mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

ndata_train = x_train.shape[0]
ndata_test = x_test.shape[0]

x_train = x_train.reshape((ndata_train,28,28,1))
x_test = x_test.reshape((ndata_test,28,28,1))


xshape = x_train.shape[1:4]

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
x_train.shape

(60000, 28, 28, 1)

In [ ]:
NNmodel = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=10,kernel_size=(5,5),input_shape=xshape),
        tf.keras.layers.Conv2D(filters=15,kernel_size=(4,4),activation=tf.nn.relu),
        tf.keras.layers.MaxPooling2D(pool_size = (3,3),strides=3),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(10,activation=tf.nn.softmax)
        ])

In [ ]:
NNmodel.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

NNmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 10)        260       
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 15)        2415      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 7, 15)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 735)               0         
                                                                 
 dense (Dense)               (None, 64)                47104     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                        

In [ ]:
NNmodel.fit(x_train,y_train,epochs=10,validation_split=0.1,batch_size=100)

Epoch 1/10
540/540 [==============================] - 43s 79ms/step - loss: 0.5034 - accuracy: 0.8205 - val_loss: 0.3744 - val_accuracy: 0.8628
Epoch 2/10
540/540 [==============================] - 43s 79ms/step - loss: 0.3890 - accuracy: 0.8605 - val_loss: 0.3379 - val_accuracy: 0.8740
Epoch 3/10
540/540 [==============================] - 42s 78ms/step - loss: 0.3449 - accuracy: 0.8767 - val_loss: 0.3113 - val_accuracy: 0.8845
Epoch 4/10
540/540 [==============================] - 43s 79ms/step - loss: 0.3182 - accuracy: 0.8858 - val_loss: 0.2979 - val_accuracy: 0.8903
Epoch 5/10
540/540 [==============================] - 42s 78ms/step - loss: 0.3020 - accuracy: 0.8911 - val_loss: 0.2857 - val_accuracy: 0.8967
Epoch 6/10
540/540 [==============================] - 42s 78ms/step - loss: 0.2872 - accuracy: 0.8961 - val_loss: 0.2710 - val_accuracy: 0.9002
Epoch 7/10
540/540 [==============================] - 42s 78ms/step - loss: 0.2756 - accuracy: 0.9011 - val_loss: 0.2678 - val_accuracy:

In [ ]:
pred_probs = NNmodel.predict(x_test)
pred = np.argmax(pred_probs, axis=1)
print(np.mean(pred==y_test))

0.8991


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# problem 3
data = pd.read_csv('/content/drive/MyDrive/optimization_colabs/TimeOfTravel.csv')
obj = np.ones(52)/51
obj[0] = 0

A = np.zeros((102,52))
rhs = np.zeros(102)
direction = np.array(['>']*102)
lb = np.array([-np.inf]*52)
lb[0] = 0

for row in range(51):
    A[2*row,[0,row+1]] = [1,1] # h_i >= T_i - d -> hi + d >= Ti
    rhs[2*row] = data.travel_time[row]
    A[2*row+1,[0,row+1]] = [-1,1] # h_i >= d - T_i -> hi - d >= -Ti
    rhs[2*row+1] = -data.travel_time[row]



In [ ]:
spMod = gp.Model()
spMod_x = spMod.addMVar(len(obj),lb=lb) # tell the model how many variables there are
# must define the variables before adding constraints because variables go into the constraints
spMod_con = spMod.addMConstrs(A, spMod_x, direction, rhs) # NAME THE CONSTRAINTS!!! so we can get information about them later!
spMod.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE) # add the objective to the model...we'll talk about the None and the 0

spMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
spMod.optimize()

spMod_x.x[0]

44.0

In [ ]:
np.median(data.travel_time) # this is NOT a coincidence!

44.0

In [ ]:
# problem 4
# epsilon greedy